<img src="../Images/DSC_Logo.png" style="width: 400px;">

# Time Series Theory in Python - Part 4: Regression and Advanced Models

This notebook demonstrates time series regression and introduces exemplary advanced modeling techniques that can be advantageous for time series prediction.

In [ ]:
pip install statsmodels PythonTsa statsmodels scikit-learn openpyxl tensorflow

In [ ]:
from PythonTsa.datadir import getdtapath
dtapath=getdtapath()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

## 1 Time Series Regression Techniques

In [ ]:
import statsmodels.api as sm

### **Example 1: Australian Employed Total Persons**

In [ ]:
# Get the data file path
dtapath = getdtapath()

# Load the Excel file containing employment data
aul = pd.read_excel(dtapath + 'AustraliaEmployedTotalPersons.xlsx', header=0)

# Create a time index starting from February 1978 with monthly frequency
timeindex = pd.date_range('1978-02', periods=len(aul), freq='ME')
aul.index = timeindex

# Extracting employed persons
y = aul['EmployedP'].values
time = np.arange(len(y))  # Create an array of time periods (0, 1, 2, ...)

# Fit OLS models
linear_model = sm.OLS(y, sm.add_constant(time)).fit()
quadratic_model = sm.OLS(y, sm.add_constant(np.column_stack((time, time**2)))).fit()
cubic_model = sm.OLS(y, sm.add_constant(np.column_stack((time, time**2, time**3)))).fit()

# Extracting the slope and intercept for the linear model
slope_LSTM = linear_model.params[1]  
intercept_LSTM = linear_model.params[0] 
print("m:", slope_LSTM, "n:", intercept_LSTM)

# Plotting the results
plt.figure()
plt.scatter(aul.index, y, label='Total Employed Persons', color='gray', alpha=0.5)

# Fitted values from each model
plt.plot(aul.index, linear_model.fittedvalues, label='Linear', color='blue')
plt.plot(aul.index, quadratic_model.fittedvalues, label='Quadratic', color='black')
plt.plot(aul.index, cubic_model.fittedvalues, label='Cubic', color='red', linestyle='--')

# Adding titles and labels
plt.title('Trend Analysis of Total Employed Persons in Australia')
plt.xlabel('Date')
plt.ylabel('Number of Persons Employed')
plt.legend()
plt.grid()
plt.show()

# Print the summary statistics for models
print("Linear Model Summary:")
print(linear_model.summary())
print("\nQuadratic Model Summary:")
print(quadratic_model.summary())
print("\nCubic Model Summary:")
print(cubic_model.summary())

Compare models using AIC and the adjusted R-squared:

In [ ]:
print("\nModel Comparison:")
print(f"Linear Model - AIC: {linear_model.aic:.2f}, Adjusted R-squared: {linear_model.rsquared_adj:.4f}")
print(f"Quadratic Model - AIC: {quadratic_model.aic:.2f}, Adjusted R-squared: {quadratic_model.rsquared_adj:.4f}")
print(f"Cubic Model - AIC: {cubic_model.aic:.2f}, Adjusted R-squared: {cubic_model.rsquared_adj:.4f}")

### **Example 2: Mean Spring Passage Dates of European Pied Flycatcher**

The dataset contains the migration data of the European Pied Flycatcher, focusing on the adjusted mean spring passage dates (MADJDAYSWS) across multiple years. The dataset includes annual observations, capturing trends in migration timing and the relationship between migration timing and several influential climate variables from key geographic regions.

Identified final weather variables that are likely to affect mean spring passage dates at Helgoland of the European Pied Flycatcher:
- ID14: mean temperature in a specific region Germany (23 Mar - 14 May)
- ID25: number of days with winds coming from Helgoland in a region in Mali (09 Feb - 09 Jun)
- ID15: mean temperature in a specific region in Italy (27 Nov - 12 Dec)
- ID8: mean temperature in the region of northwestern Algeria and northeastern Marocco (10 Feb - 08 Jun)
- ID53: number of days with winds going to Helgoland in a region in Nigeria (16 Apr - 05 May)

**Original dataset and code:** Haest, B., Hüppop, O., & Bairlein, F. (2020). Code and data for: "Weather at the winter and stopover areas determines spring migration onset, progress, and advancements in Afro-Palearctic migrant birds". In Proceedings of the National Academy of Sciences of the United States of America (v1.0, Bd. 117, Nummer 29, S. 17056–17062). Zenodo. doi:10.5281/zenodo.3629178              

**Related publication(s):** Haest, B., Hüppop, O., and Bairlein, F.: Weather at the winter and stopover areas determines spring migration onset, progress, and advancements in Afro-Palearctic migrant birds, Proceedings of the National Academy of Sciences, 117, 17056–17062, doi:10.1073/pnas.1920448117,              2020.

Original data and code were modified for this notebook.

In [ ]:
bird_migration = pd.read_csv('../Datasets/bird_migration.csv', sep = ';')
bird_migration.head()

The study fitted regression lines to the mean spring passage dates, modeling the trend in MSPD using just the temporal aspect (e.g., the effect of the year) to understand how the MSPD changes over time.

In [ ]:
# Assuming the 'MADJDAYSWS' is the column you want as y
y2 = bird_migration['MADJDAYSWS'].values  # Extracting dependent variable
time2 = bird_migration['Year'].values  # Using the Year column as the time variable

# Fit OLS models
linear_model = sm.OLS(y2, sm.add_constant(time2)).fit()
quadratic_model = sm.OLS(y2, sm.add_constant(np.column_stack((time2, time2**2)))).fit()
cubic_model = sm.OLS(y2, sm.add_constant(np.column_stack((time2, time2**2, time2**3)))).fit()

# Extracting the slope and intercept for the linear model
slope_LSTM = linear_model.params[1]  
intercept_LSTM = linear_model.params[0] 
print("m:", slope_LSTM, "n:", intercept_LSTM)

# Plotting the results
plt.figure()
plt.scatter(bird_migration['Year'], y2, label='Data')
plt.plot(time2, linear_model.fittedvalues, label='Linear', color='blue')
plt.plot(time2, quadratic_model.fittedvalues, label='Quadratic', color='black')
plt.plot(time2, cubic_model.fittedvalues, label='Cubic', color='red', linestyle='--')
plt.legend()
plt.xlabel('Year')
plt.ylabel('MADJDAYSWS')
plt.title('Trend Analysis for European Pied Flycatcher')
plt.show()

When comparing models for the mean spring passage dates (MSPD) of migratory birds, AIC helps identify which trend model balances goodness-of-fit with simplicity, where lower values suggest a more efficient model. Additionally, the adjusted R-squared provides information how well the model explains the variance in the data.

In [ ]:
print("\nModel Comparison:")
print(f"Linear Model - AIC: {linear_model.aic:.2f}, Adjusted R-squared: {linear_model.rsquared_adj:.4f}")
print(f"Quadratic Model - AIC: {quadratic_model.aic:.2f}, Adjusted R-squared: {quadratic_model.rsquared_adj:.4f}")
print(f"Cubic Model - AIC: {cubic_model.aic:.2f}, Adjusted R-squared: {cubic_model.rsquared_adj:.4f}")

Birds arrive earlier. The linear model yielded the lowest AIC of 329.38, indicating it is the best fit among the three models, despite the relatively low adjusted R-squared of 0.3311, suggesting that approximately 33% of the variability in the migration timing is explained by the model. 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance

# Extracting the dependent variable and the features
y2 = bird_migration['MADJDAYSWS'].values  # Target variable
X = bird_migration[['ID14', 'ID25', 'ID15', 'ID8', 'ID53']]  # Features

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.2, random_state=42)

# Train a Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predict on the test set
y_pred = linear_model.predict(X_test)

# Print model coefficients
print("Model coefficients:")
for i, col in enumerate(X.columns):
    print(f"{col}: {linear_model.coef_[i]}")

# Calculate permutation feature importance
result = permutation_importance(linear_model, X_test, y_test, n_repeats=30, random_state=42)

# Get feature importance
importance = result.importances_mean
indices = np.argsort(importance)[::-1]

# Print feature ranking
print("Feature ranking:")
for i in range(X.shape[1]):
    print(f"{i + 1}. Feature {X.columns[indices[i]]} ({importance[indices[i]]})")

# Visualize feature importance
plt.figure()
plt.title("Permutation Feature Importance")
plt.bar(range(X.shape[1]), importance[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.ylabel("Importance")
plt.xlabel("Features")
plt.show()

Interpretation:
- ID25: Mali (Wind - Coming From Helgoland, 09 Feb - 09 Jun): Identified as the most important factor for predicting migration timing, with a feature importance score of 0.1162. Good winds from Helgoland at the wintering site help the birds travel faster, making them arrive earlier. Although we did not apply exactly the same methods as the study, this finding aligns with the study's finding that wind conditions, particularly in stopover and wintering sites, are a major influence on spring migration timing.
- ID14: Germany (Temperature, 23 Mar - 14 May): The temperature during this time affects when birds migrate, suggesting that warmer temperatures lead to earlier migrations.
- ...

## 2. Advanced Time Series Prediction Techniques

## 2.1 GAM

$$E(Y)=β0​+f1​(X1​)+f2​(X2​)+...+fk​(Xk​)$$

### **Example 1 [continued]: Global Annual Mean Surface Air Temperature Changes Series (1880-1985)**

The time series dataset "Global mean surface air temperature changes 1880–1985" (denoted as GMSATC) from the folder `Ptsadata` is from Hansen and Lebedeff (1987) that investigates the global warming issue. 

In [ ]:
import statsmodels.api as sm
from patsy import dmatrix

# Load the Excel file containing employment data
dtapath = getdtapath()
aul = pd.read_excel(dtapath + 'AustraliaEmployedTotalPersons.xlsx', header=0)

# Create a time index starting from February 1978 with monthly frequency
timeindex = pd.date_range('1978-02', periods=len(aul), freq='ME')
aul.index = timeindex

# Extracting employed persons
y = aul['EmployedP'].values
time = np.arange(len(y))  # Create an array of time periods (0, 1, 2, ...)

# Fit OLS models
linear_model = sm.OLS(y, sm.add_constant(time)).fit()
quadratic_model = sm.OLS(y, sm.add_constant(np.column_stack((time, time**2)))).fit()
cubic_model = sm.OLS(y, sm.add_constant(np.column_stack((time, time**2, time**3)))).fit()

# Fit a Generalized Additive Model (GAM) using B-splines
bsplines = dmatrix("bs(time, df=6)", {"time": time}, return_type='dataframe')
gam_model = sm.GLM(y, bsplines, family=sm.families.Gaussian()).fit()

# Plotting results
plt.figure()
plt.scatter(aul.index, y, label='Total Employed Persons', color='gray', alpha=0.5)

# Fitted values from each model
plt.plot(aul.index, linear_model.fittedvalues, label='Linear', color='blue')
plt.plot(aul.index, quadratic_model.fittedvalues, label='Quadratic', color='black')
plt.plot(aul.index, cubic_model.fittedvalues, label='Cubic', color='red', linestyle='--')
plt.plot(aul.index, gam_model.fittedvalues, label='GAM', color='green', linewidth=2)

# Adding titles and labels
plt.title('Trend Analysis of Total Employed Persons in Australia')
plt.xlabel('Date')
plt.ylabel('Number of Persons Employed')
plt.legend()
plt.grid()
plt.show()

# Print the summary statistics for models
print("Linear Model Summary:")
print(linear_model.summary())
print("\nQuadratic Model Summary:")
print(quadratic_model.summary())
print("\nCubic Model Summary:")
print(cubic_model.summary())
print("\nGAM Model Summary:")
print(gam_model.summary())

## 2.2 Neural Networks

### **Example 2: Hourly Series of Electricity Load**

## 2.2.1 Dataset Preprocessing

Preprocessing datasets for machine learning tasks generally involves scaling and splitting the dataset. Datasets are typically divided into three parts: the train set, validation set, and test set. These sets do not overlap. This approach ensures a robust evaluation of the performance of a machine learning model. A two-split approach (training and validation) can be sufficient depending on the research question. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_percentage_error

# Load the data
tsdta = pd.read_csv(dtapath + "elec-temp.csv")
tsdta['time'] = pd.to_datetime(tsdta['time'])
tsdta.set_index('time', inplace=True)

# Ensure we are only working with the "load" data
loadts = tsdta[['load']]  # Assuming the column is named 'load'

# Plot for loadts2
loadts2 = loadts[loadts.index > '2014-12-18 00:00:00']
loadts2.plot(title='Load from 2014-12-18 00:00:00 to present')
plt.xlabel('Time')
plt.ylabel('Load')
plt.show()

# Plot for loadts3
loadts3 = loadts[(loadts.index > '2012-03-01 00:00:00') & (loadts.index < '2014-03-01 00:00:00')]
loadts3.plot(title='Load from 2012-03-01 00:00:00 to 2014-03-01 00:00:00')
plt.xlabel('Time')
plt.ylabel('Load')
plt.show()


# Splitting the datasets
validtime = '2014-09-01 00:00:00'
testtime = '2014-11-01 00:00:00'

loadts_split = (
    loadts[(loadts.index < validtime)].rename(columns={'load': 'train'})
    .join(loadts[(loadts.index >= validtime) & (loadts.index < testtime)]
           .rename(columns={'load': 'validation'}), how='outer')
    .join(loadts[testtime:].rename(columns={'load': 'test'}), how='outer')
)

# Plotting the train, validation, and test sets
loadts_split.plot(y=['train', 'validation', 'test'], style=['-', '-.', '--'])
plt.ylabel('Electricity load')
plt.title('Load Data Train, Validation and Test Split')
plt.show()

# Train/Validation/Test datasets
train = loadts.copy()[loadts.index < validtime]
valid = loadts.copy()[(loadts.index >= validtime) & (loadts.index < testtime)]
test = loadts.copy()[loadts.index >= testtime]

# Scaling the data
scaler = MinMaxScaler()
train['load'] = scaler.fit_transform(train)
valid['load'] = scaler.transform(valid)
test['load'] = scaler.transform(test)

# ACF
plot_acf(train['load'], lags=72)
plt.title('ACF of the Generated AR(2) Sample')
plt.show()

In this preprocessing procedure, the dataset is manually split into training, validation, and test sets based on specific timestamps. This method is especially usefull for time series, because it retains the temporal ordering of the data. For specific tasks, the `train_test_split` function can also be employed to randomly partition the dataset into training and testing subsets:

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming loadts is your DataFrame with a 'load' column
# Splitting the dataset randomly into train and test
train, temp = train_test_split(loadts, test_size=0.3, random_state=42)  # 70% train and 30% temp
valid, test = train_test_split(temp, test_size=0.5, random_state=42)  # Split temp into 50% valid and 50% test

# Plotting the train, validation, and test sets
plt.figure()
plt.scatter(train.index, train['load'], label='Train')
plt.scatter(valid.index, valid['load'], label='Validation')
plt.scatter(test.index, test['load'], label='Test')
plt.ylabel('Electricity load')
plt.title('Load Data Train, Validation and Test Split')
plt.legend()
plt.show()

# Scaling the data
scaler = MinMaxScaler()
train['load'] = scaler.fit_transform(train[['load']])
valid['load'] = scaler.transform(valid[['load']])
test['load'] = scaler.transform(test[['load']])

## 2.2.2 Build Gated Recurrent Unit (GRU) Model

In [ ]:
# Preparing the training dataset for the GRU model
T = 24  # Lookback period
HORIZON = 1  # One-step-ahead prediction

train_shifted = train.copy()
train_shifted['y_t+1'] = train_shifted['load'].shift(-1, freq='H')

for t in range(1, T + 1):
    train_shifted[str(T - t)] = train_shifted['load'].shift(T - t, freq='H')

y_col = 'y_t+1'
X_cols = [f'load_t-{i}' for i in range(23, -1, -1)]
train_shifted.columns = ['load_original'] + [y_col] + X_cols
train_shifted = train_shifted.dropna(how='any')

y_train = np.array(train_shifted[y_col])
X_train = np.array(train_shifted[X_cols])
X_train = X_train.reshape(X_train.shape[0], T, 1)

# Preparing validation and test sets (similar to train_shifted)
valid_shifted = valid.copy()
test_shifted = test.copy()

for data_shifted, dataset in zip([valid_shifted, test_shifted], [valid, test]):
    data_shifted['y_t+1'] = data_shifted['load'].shift(-1, freq='H')
    for t in range(1, T + 1):
        data_shifted[str(T - t)] = data_shifted['load'].shift(T - t, freq='H')
    data_shifted.columns = ['load_original'] + [y_col] + X_cols
    data_shifted.dropna(how='any', inplace=True)

# Prepare y and X for validation and test datasets
y_valid, y_test = np.array(valid_shifted[y_col]), np.array(test_shifted[y_col])
X_valid = np.array(valid_shifted[X_cols]).reshape(-1, T, 1)
X_test = np.array(test_shifted[X_cols]).reshape(-1, T, 1)

# Building the GRU model
latent_dim = 6
batch_size = 32
epochs = 15

model_GRU = Sequential()
model_GRU.add(GRU(latent_dim, input_shape=(T, 1)))
model_GRU.add(Dense(HORIZON))
model_GRU.compile(optimizer='RMSprop', loss='mse')
model_GRU.summary()

# Training the model
model_GRU.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

## 2.2.3 Predict

In [ ]:
# Assuming you generate predictions called `preds`
preds = model.predict(X_test)

# Create DataFrame for evaluation
evdta = pd.DataFrame(preds, columns=[f't+{t}' for t in range(1, HORIZON + 1)])
evdta['time'] = test_shifted.index
evdta = pd.melt(evdta, id_vars='time', value_name='fitted', var_name='h')
evdta['actual'] = np.transpose(y_test).ravel()
evdta[['fitted', 'actual']] = scaler.inverse_transform(evdta[['fitted', 'actual']])

# Error calculation
error = mean_absolute_percentage_error(evdta['actual'], evdta['fitted'])
print(f'Mean Absolute Percentage Error: {error}')

# Plotting the fitted vs actual values
evdta[evdta.time < '2014-11-08'].plot(x='time', y=['fitted', 'actual'], style=['--r', '-b'])
plt.ylabel('Electricity load')
plt.title('Fitted vs Actual Load Values')
plt.show()

## 2.2.4 Another NN model: Long-Short-Term Memory (LSTM) 

To build a first version of another type of Neural Network model (now LSTM models), simply change the following part. In this adjusted code, we replace the GRU layer with an LSTM layer. While both GRUs and LSTMs are types of recurrent neural networks and are effective for sequence prediction problems, LSTMs have the advantage of being able to remember longer sequences due to their architecture. We will not continue with predictions using the LSTM model at this time, as it requires further hyperparameter tuning and experimentation to optimize performance before we can reliably compare its results with the previously established GRU model.

In [ ]:
from tensorflow.keras.layers import LSTM

model_LSTM = Sequential()
model_LSTM.add(LSTM(latent_dim, input_shape=(T, 1)))
model_LSTM.add(Dense(HORIZON))
model_LSTM.compile(optimizer='RMSprop', loss='mse')
model_LSTM.summary()